##Spark implements a subset of ANSI SQL:2003.
With the release of Spark 2.0, its authors created a superset of Hive’s support, writing a native
SQL parser that supports both ANSI-SQL as well as HiveQL queries

Spark provides a Java Database Connectivity (JDBC) interface by which either you or a remote
program connects to the Spark driver in order to execute Spark SQL queries. A common use case
might be a for a business analyst to connect business intelligence software like Tableau to Spark.
The Thrift JDBC/Open Database Connectivity (ODBC) server implemented here corresponds to
the HiveServer2 in Hive 1.2.1. You can test the JDBC server with the beeline script that comes
with either Spark or Hive 1.2.1.
To start the JDBC/ODBC server, run the following in the Spark directory:
./sbin/start-thriftserver.sh
This script accepts all bin/spark-submit command-line options. To see all available options
for configuring this Thrift Server, run ./sbin/start-thriftserver.sh --help. By default,
the server listens on localhost:10000. You can override this through environmental variables or
system properties.

The core difference between tables and DataFrames is this: you define
DataFrames in the scope of a programming language, whereas you define tables within a
database. This means that when you create a table (assuming you never changed the database), it
will belong to the default database.

An important thing to note is that in Spark 2.X, tables always contain data. There is no notion of
a temporary table, only a view, which does not contain data. This is important because if you go
to drop a table, you can risk losing the data when doing so.

The data within the tables as well as the data about the tables;
that is, the metadata. You can have Spark manage the metadata for a set of files as well as for the
data.

When you define a table from files on disk, you are defining an unmanaged table. When
you use saveAsTable on a DataFrame, you are creating a managed table for which Spark will
track of all of the relevant information.

This will read your table and write it out to a new location in Spark format. You can see this
reflected in the new explain plan. In the explain plan, you will also notice that this writes to the
default Hive warehouse location. You can set this by setting the spark.sql.warehouse.dir
configuration to the directory of your choosing when you create your SparkSession. By default
Spark sets this to /user/hive/warehouse:

Note in the results that a database is listed. Spark also has databases which we will discuss later
in this chapter, but for now you should keep in mind that you can also see tables in a specific
database by using the query show tables IN databaseName, where databaseName represents
the name of the database that you want to query.
If you are running on a new cluster or local mode, this should return zero results.

In [0]:
table = spark.sql('''CREATE TABLE flights (
DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
USING JSON OPTIONS (path 'dbfs:/FileStore/flight_data/2015_summary.json')''')

In [0]:
type(table)

Out[3]: pyspark.sql.dataframe.DataFrame

In [0]:
table.show(10)

++
||
++
++



In [0]:
spark.sql("Select * from flights limit 2").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+



In [0]:
spark.sql('''CREATE TABLE flights_dummy
USING JSON OPTIONS (path 'dbfs:/FileStore/flight_data/2015_summary.json')''')

Out[9]: DataFrame[]

In [0]:
spark.sql("Select * from flights_dummy limit 2").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+



In [0]:
table = spark.sql('''CREATE TABLE flights_csv (
DEST_COUNTRY_NAME STRING, 
ORIGIN_COUNTRY_NAME STRING COMMENT "remember, the US will be most prevalent", 
count LONG)
USING JSON OPTIONS (path 'dbfs:/FileStore/flight_data/2015_summary.json')''')

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-61910737769275> in <cell line: 1>()
----> 1 table = spark.sql('''CREATE TABLE flights_csv (
      2 DEST_COUNTRY_NAME STRING,
      3 ORIGIN_COUNTRY_NAME STRING COMMENT "remember, the US will be most prevalent",
      4 count LONG)
      5 USING JSON OPTIONS (path 'dbfs:/FileStore/flight_data/2015_summary.json')''')

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/session.py in sql(self, sqlQuery, **kwargs)
   1117             sqlQuery = formatter.format(sqlQuery, **kwargs)
   1118 

In [0]:
#Describe table metadata
spark.sql("DESCRIBE TABLE flights_csv").show()

+-------------------+---------+--------------------+
|           col_name|data_type|             comment|
+-------------------+---------+--------------------+
|  DEST_COUNTRY_NAME|   string|                null|
|ORIGIN_COUNTRY_NAME|   string|remember, the US ...|
|              count|   bigint|                null|
+-------------------+---------+--------------------+



In [0]:
spark.sql("Select * from flights_csv limit 2").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+



In [0]:
#It is possible to create a table from a query as well:
spark.sql('''
CREATE TABLE flights_from_select USING parquet AS SELECT * FROM flights
''')

Out[17]: DataFrame[]

In [0]:
spark.sql("Select * from flights_from_select limit 2").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+



In [0]:
# DROP TABLE IF EXISTS
spark.sql('''
Drop TABLE flights_from_select
''')

Out[16]: DataFrame[]

Dropping unmanaged tables
If you are dropping an unmanaged table (e.g., hive_flights), no data will be removed but you
will no longer be able to refer to this data by the table name.

In [0]:
#you can control the layout of the data by writing out a partitioned dataset,
spark.sql('''
CREATE TABLE partitioned_flights USING parquet PARTITIONED BY (DEST_COUNTRY_NAME)
AS SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights 
''')

Out[19]: DataFrame[]

In [0]:
spark.sql("SHOW PARTITIONS partitioned_flights").show(20,False)

+---------------------------------------------------+
|partition                                          |
+---------------------------------------------------+
|DEST_COUNTRY_NAME=Algeria                          |
|DEST_COUNTRY_NAME=Angola                           |
|DEST_COUNTRY_NAME=Anguilla                         |
|DEST_COUNTRY_NAME=Antigua and Barbuda              |
|DEST_COUNTRY_NAME=Argentina                        |
|DEST_COUNTRY_NAME=Aruba                            |
|DEST_COUNTRY_NAME=Australia                        |
|DEST_COUNTRY_NAME=Austria                          |
|DEST_COUNTRY_NAME=Azerbaijan                       |
|DEST_COUNTRY_NAME=Bahrain                          |
|DEST_COUNTRY_NAME=Barbados                         |
|DEST_COUNTRY_NAME=Belgium                          |
|DEST_COUNTRY_NAME=Belize                           |
|DEST_COUNTRY_NAME=Bermuda                          |
|DEST_COUNTRY_NAME=Bolivia                          |
|DEST_COUNTRY_NAME=Bonaire, 

These tables will be available in Spark even through sessions; temporary tables do not currently
exist in Spark. You must create a temporary view, which we demonstrate later in this chapter.

Insertion:

INSERT INTO flights_from_select
SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights LIMIT 20
You can optionally provide a partition specification if you want to write only into a certain
partition. Note that a write will respect a partitioning scheme, as well (which may cause the
above query to run quite slowly); however, it will add additional files only into the end
partitions:
INSERT INTO partitioned_flights
PARTITION (DEST_COUNTRY_NAME="UNITED STATES")
SELECT count, ORIGIN_COUNTRY_NAME FROM flights
WHERE DEST_COUNTRY_NAME='UNITED STATES' LIMIT 12

REFRESH TABLE refreshes
all cached entries (essentially, files) associated with the table. If the table were previously
cached, it would be cached lazily the next time it is scanned:

In [0]:
spark.sql("REFRESH table flights_csv")

Out[7]: DataFrame[]

Another related command is REPAIR TABLE, which refreshes the partitions maintained in the
catalog for that given table. This command’s focus is on collecting new partition information—
an example might be writing out a new partition manually and the need to repair the table
accordingly:
MSCK REPAIR TABLE partitioned_flights

Just like DataFrames, you can cache and uncache tables. You simply specify which table you
would like using the following syntax:

CACHE TABLE flights

Here’s how you uncache them:

UNCACHE TABLE FLIGHTS

view specifies a set
of transformations on top of an existing table—basically just saved query plans, which can be
convenient for organizing or reusing your query logic. Views can be global, set to a database, or per session.

To an end user, views are displayed as tables, except rather than rewriting all of the data to a new
location, they simply perform a transformation on the source data at query time. This might be a
filter, select, or potentially an even larger GROUP BY or ROLLUP. For instance, in the
following example, we create a view in which the destination is United States in order to see
only those flights

In [0]:
spark.sql('''
CREATE VIEW just_usa_view AS
SELECT * FROM flights WHERE dest_country_name = 'United States'
''')

Out[8]: DataFrame[]

In [0]:
spark.sql("Select * from just_usa_view limit 5").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|    United States|              India|   62|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+



Like tables, you can create temporary views that are available only during the current session and
are not registered to a database:

CREATE TEMP VIEW just_usa_view_temp AS
SELECT * FROM flights WHERE dest_country_name = 'United States'

Or, it can be a global temp view. Global temp views are resolved regardless of database and are
viewable across the entire Spark application, but they are removed at the end of the session:

CREATE GLOBAL TEMP VIEW just_usa_global_view_temp AS
SELECT * FROM flights WHERE dest_country_name = 'United States'

You can also specify that you would like to overwite a view if one already exists by using the
keywords shown in the sample that follows. We can overwrite both temp views and regular
views:

CREATE OR REPLACE TEMP VIEW just_usa_view_temp AS

SELECT * FROM flights WHERE dest_country_name = 'United States'

Effectively, views are equivalent to creating a new DataFrame from an existing DataFrame

In [0]:
spark.sql("DROP VIEW IF EXISTS just_usa_view")

Out[10]: DataFrame[]

Databases are a tool for organizing tables. As mentioned earlier, if you do not define one, Spark
will use the default database. Any SQL statements that you run from within Spark (including
DataFrame commands) execute within the context of a database. This means that if you change
the database, any user-defined tables will remain in the previous database and will need to be
queried differently.

This can be a source of confusion, especially if you’re sharing the same context or session for your
coworkers, so be sure to set your databases appropriately

In [0]:
spark.sql("SHOW DATABASES").show()

+------------+
|databaseName|
+------------+
|     default|
+------------+



In [0]:
spark.sql("CREATE DATABASE demo_db")
#to use a specific db
spark.sql("USE demo_db")

Out[14]: DataFrame[]

In [0]:
spark.sql("Show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [0]:
#spark.sql("SELECT * FROM flights limit 5").show() # should fail
spark.sql("SELECT * FROM default.flights limit 5").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



You can see what database you’re currently using by running the following command:

SELECT current_database()

You can, of course, switch back to the default database:

USE default;

In [0]:
spark.sql("DROP DATABASE IF EXISTS demo_db;")

Out[19]: DataFrame[]

Queries in Spark support the following ANSI SQL requirements

In [0]:
spark.sql("SELECT current_database()").show()

+------------------+
|current_database()|
+------------------+
|           demo_db|
+------------------+



In [0]:
spark.sql("Use default")

Out[23]: DataFrame[]

In [0]:
spark.sql("DROP DATABASE IF EXISTS demo_db;")

Out[24]: DataFrame[]

In [0]:
#if statements:
spark.sql('''
SELECT
*, (CASE WHEN DEST_COUNTRY_NAME = 'United States' THEN 1
WHEN DEST_COUNTRY_NAME = 'Egypt' THEN 0
ELSE -1 END) as condition
FROM flights
''').show(10)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|condition|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
|    United States|            Ireland|  344|        1|
|            Egypt|      United States|   15|        0|
|    United States|              India|   62|        1|
|    United States|          Singapore|    1|        1|
|    United States|            Grenada|   62|        1|
|       Costa Rica|      United States|  588|       -1|
|          Senegal|      United States|   40|       -1|
|          Moldova|      United States|    1|       -1|
+-----------------+-------------------+-----+---------+
only showing top 10 rows



Complex types are a departure from standard SQL and are an incredibly powerful feature that
does not exist in standard SQL. Understanding how to manipulate them appropriately in SQL is
essential. There are three core complex types in Spark SQL: structs, lists, and maps.

Structs are more akin to maps. They provide a way of creating or querying nested data in Spark.
To create one, you simply need to wrap a set of columns (or expressions) in parentheses:

In [0]:
spark.sql('''CREATE VIEW IF NOT EXISTS nested_data AS
SELECT (DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME) as country, count FROM flights''')

Out[30]: DataFrame[]

In [0]:
spark.sql(''' select * from nested_data''').show(5,False)

+------------------------+-----+
|country                 |count|
+------------------------+-----+
|{United States, Romania}|15   |
|{United States, Croatia}|1    |
|{United States, Ireland}|344  |
|{Egypt, United States}  |15   |
|{United States, India}  |62   |
+------------------------+-----+
only showing top 5 rows



In [0]:
spark.sql(''' select country.DEST_COUNTRY_NAME, count from nested_data''').show(5,False)

+-----------------+-----+
|DEST_COUNTRY_NAME|count|
+-----------------+-----+
|United States    |15   |
|United States    |1    |
|United States    |344  |
|Egypt            |15   |
|United States    |62   |
+-----------------+-----+
only showing top 5 rows



You can use the collect_list function,
which creates a list of values. You can also use the function collect_set, which creates an
array without duplicate values. These are both aggregation functions

In [0]:
spark.sql('''
SELECT DEST_COUNTRY_NAME as new_name, collect_list(count) as flight_counts,
collect_set(ORIGIN_COUNTRY_NAME) as origin_set
FROM flights GROUP BY DEST_COUNTRY_NAME
''').show(5)

+-------------------+-------------+---------------+
|           new_name|flight_counts|     origin_set|
+-------------------+-------------+---------------+
|            Algeria|          [4]|[United States]|
|             Angola|         [15]|[United States]|
|           Anguilla|         [41]|[United States]|
|Antigua and Barbuda|        [126]|[United States]|
|          Argentina|        [180]|[United States]|
+-------------------+-------------+---------------+
only showing top 5 rows



You can, however, also create an array manually within a column, as shown here:
SELECT DEST_COUNTRY_NAME, ARRAY(1, 2, 3) FROM flights

You can also query lists by position by using a Python-like array query syntax:

SELECT DEST_COUNTRY_NAME as new_name, collect_list(count)[0]

FROM flights GROUP BY DEST_COUNTRY_NAME

You can also do things like convert an array back into rows. You do this by using the explode
function.

In [0]:
spark.sql('''
CREATE OR REPLACE TEMP VIEW flights_agg AS
SELECT DEST_COUNTRY_NAME, collect_list(count) as collected_counts
FROM flights GROUP BY DEST_COUNTRY_NAME
''')

Out[42]: DataFrame[]

In [0]:
spark.sql('''
SELECT explode(collected_counts), DEST_COUNTRY_NAME FROM flights_agg
''').show(5)

+---+-------------------+
|col|  DEST_COUNTRY_NAME|
+---+-------------------+
|  4|            Algeria|
| 15|             Angola|
| 41|           Anguilla|
|126|Antigua and Barbuda|
|180|          Argentina|
+---+-------------------+
only showing top 5 rows



In [0]:
spark.sql("SHOW FUNCTIONS").show()

+--------+
|function|
+--------+
|       !|
|      !=|
|       %|
|       &|
|       *|
|       +|
|       -|
|       /|
|       <|
|      <=|
|     <=>|
|      <>|
|       =|
|      ==|
|       >|
|      >=|
|       ^|
|     abs|
|    acos|
|   acosh|
+--------+
only showing top 20 rows



In [0]:
spark.sql("SHOW SYSTEM FUNCTIONS").show()

+--------+
|function|
+--------+
|       !|
|      !=|
|       %|
|       &|
|       *|
|       +|
|       -|
|       /|
|       <|
|      <=|
|     <=>|
|      <>|
|       =|
|      ==|
|       >|
|      >=|
|       ^|
|     abs|
|    acos|
|   acosh|
+--------+
only showing top 20 rows



In [0]:
spark.sql("SHOW USER FUNCTIONS").show()

+--------+
|function|
+--------+
+--------+



In [0]:
spark.sql("SHOW FUNCTIONS 's*'").show()

+------------------+
|          function|
+------------------+
|     schema_of_csv|
|    schema_of_json|
|               sec|
|            second|
|            secret|
|         sentences|
|          sequence|
|    session_window|
|               sha|
|              sha1|
|              sha2|
|         shiftleft|
|        shiftright|
|shiftrightunsigned|
|           shuffle|
|              sign|
|            signum|
|               sin|
|              sinh|
|              size|
+------------------+
only showing top 20 rows



In [0]:
spark.sql("DESCRIBE FUNCTION 'SEQUENCE'").show(5,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|function_desc                                                                                                                                                                                                                                                                                                                                                        

In [0]:

power3 = lambda number : number * number * number
spark.udf.register("power3", power3)

Out[15]: <function __main__.<lambda>(number)>

In [0]:
spark.sql("SHOW USER FUNCTIONS").show()

+--------+
|function|
+--------+
|  power3|
+--------+



In [0]:
spark.sql("SELECT count, power3(count) FROM flights").show(5)

+-----+-------------+
|count|power3(count)|
+-----+-------------+
|   15|         3375|
|    1|            1|
|  344|     40707584|
|   15|         3375|
|   62|       238328|
+-----+-------------+
only showing top 5 rows



Correlated subqueries use some information from the outer scope of the query in
order to supplement information in the subquery. Uncorrelated subqueries include no
information from the outer scope. Each of these queries can return one (scalar subquery) or more
values. Spark also includes support for predicate subqueries, which allow for filtering based on
values.

In [0]:
spark.sql('''
SELECT dest_country_name FROM flights
GROUP BY dest_country_name ORDER BY sum(count) DESC LIMIT 5
''').show()

+-----------------+
|dest_country_name|
+-----------------+
|    United States|
|           Canada|
|           Mexico|
|   United Kingdom|
|            Japan|
+-----------------+



In [0]:
# Uncorrelated predicate subqueries
spark.sql('''
SELECT * FROM flights
WHERE origin_country_name IN (SELECT dest_country_name FROM flights
GROUP BY dest_country_name ORDER BY sum(count) DESC LIMIT 5)
''').show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|            Egypt|      United States|   15|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
|           Guyana|      United States|   64|
+-----------------+-------------------+-----+
only showing top 5 rows



In [0]:
# Correlated predicate subqueries
spark.sql(
'''
SELECT * FROM flights f1
WHERE EXISTS (SELECT 1 FROM flights f2
WHERE f1.dest_country_name = f2.origin_country_name)
AND EXISTS (SELECT 1 FROM flights f2
WHERE f2.dest_country_name = f1.origin_country_name)
''').show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [0]:
# Uncorrelated scalar queries
spark.sql("SELECT *, (SELECT max(count) FROM flights) AS maximum FROM flights").show(5)

+-----------------+-------------------+-----+-------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|maximum|
+-----------------+-------------------+-----+-------+
|    United States|            Romania|   15| 370002|
|    United States|            Croatia|    1| 370002|
|    United States|            Ireland|  344| 370002|
|            Egypt|      United States|   15| 370002|
|    United States|              India|   62| 370002|
+-----------------+-------------------+-----+-------+
only showing top 5 rows

